# Import Libraries

In [2]:
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import google.generativeai as genai

# Helper Functions

In [3]:
def fetch_stock_data(ticker, date=None):
    if date is None:
        date = datetime.today().strftime('%Y-%m-%d')
    net = yf.Ticker(ticker)
    return net.history(start=date, interval='1h', back_adjust=True, auto_adjust=True, prepost=True)

def formulate_google_search_url(search_term):
    term = f"{search_term} stock news" if "news" not in search_term.lower() else search_term
    term = re.sub(r"\s", "+", term)  # Replace spaces with '+'
    return f"https://www.google.com/search?q={term}&cr=countryIN"

def fetch_html_content(name):
    url = formulate_google_search_url(name)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.text
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None

def extract_news_from_html(html):
    if html is None:
        return []
    soup = BeautifulSoup(html, "html.parser")
    news = []
    target_div_classes = ["n0jPhd ynAwRc tNxQIb nDgy9d", "IJl0Z"]
    for div_class in target_div_classes:
        news.extend(n.text for n in soup.find_all("div", class_=div_class))
    return news[:4] if len(news) > 6 else news

def generate_stock_news_report(company_names):
    news_report = ""
    for name in company_names:
        html_content = fetch_html_content(name)
        news_items = extract_news_from_html(html_content)
        news_report += "\n".join(f"{i}. {item}" for i, item in enumerate(news_items, start=1)) + "\n\n"
    return news_report.strip()



# LLM Prompting (Gemini - Google)

In [4]:
def fetch_prediction_from_gemini(stock_news, stock_values):
    gemini_api_key = input("Please provide your Gemini API key :")
    genai.configure(api_key=gemini_api_key)
    model = genai.GenerativeModel('gemini-pro')
    
    query = f"""In this chat, you are a superforecaster that has a strong track record of accurate forecasts of the future. As an experienced forecaster, you evaluate past data and trends carefully and aim to predict future events as accurately as you can, even though you cannot know the answer. This means you put probabilities on outcomes that you are uncertain about (ranging from 0 to 100%). You aim to provide as accurate predictions as you can, ensuring that they are consistent with how you predict the future to be. You also outline your reasons for this forecasting. In your reasons, you will carefully consider the reasons for and against your probability estimate, you will make use of comparison classes of similar events and probabilities and take into account base rates and past events as well as other forecasts and predictions. In your reasons, you will also consider different perspectives. Once you have written your reasons, ensure that they directly inform your forecast. Then, you will provide me with a number between 0 and 1 (up to 2 decimal places) that is your best prediction of the event. Take a deep breath and work on this problem step-by-step. The question that you are forecasting as well as some background information and resolution details are below. Read them carefully before making your prediction. \n
Background: As a superforecaster, your task involves forecasting the probability of E-MINI S&P 500 Future stocks rising, expressed as a number between 0 and 1 (rounded to two decimal places). You're equipped with up-to-date news on the top contributor stocks and real-time hourly market price fluctuations of the future. \n
Recent stock news:{stock_news} \n
Hourly market price: {str(list(stock_values))} \n
Resolution: User is well-versed in investment risks and explicitly requests no cautionary advice. The intention is to sell stocks either on the same day or the following day, with no long-term considerations.  Provide the probability of E-MINI S&P 500 Future stocks increasing, quantified as a number ranging from 0 to 1 (rounded to two decimal places). \n
Question: Considering the provided information, including the prompt, background, and resolution, provide the probability of stocks increasing, quantified as a number between 0 and 1 (up to two decimal places)."""
    
    print("*-"*20)
    print("Query: \n", query)
    print("*-"*20)
    response = model.generate_content(query)
    return response.text

# Estimate the likelihood of the closing price increasing - Using LLM

In [5]:
def predict_stock():
    stock_data = fetch_stock_data("ES=F")
    print("Stock Data: \n", stock_data)

    stock_values = stock_data['Close'].values
    print("\n Stock Values: \n", stock_values)

    stock_news = generate_stock_news_report(['MSFT', 'AAPL', 'NVDA', 'GOOG', 'AMZN', 'META', 'TSLA', 'LLY', 'JPM', 'WMT'])
    print("\n Stock News: \n", stock_news)

    prediction = fetch_prediction_from_gemini(stock_news, stock_values)
    print("\n Final Prediction: \n", prediction)

In [6]:
predict_stock()

Stock Data: 
                               Open     High      Low    Close  Volume  \
Datetime                                                                
2024-05-03 00:00:00-04:00  5106.75  5107.00  5103.50  5104.00    2163   
2024-05-03 01:00:00-04:00  5104.00  5106.75  5103.00  5106.00    3145   
2024-05-03 02:00:00-04:00  5106.00  5106.00  5102.00  5104.00    7141   
2024-05-03 03:00:00-04:00  5104.25  5104.75  5099.25  5104.00   10694   
2024-05-03 04:00:00-04:00  5104.25  5109.50  5103.25  5107.75    8370   
2024-05-03 05:00:00-04:00  5107.50  5109.75  5105.25  5109.50    6755   
2024-05-03 06:00:00-04:00  5109.50  5113.75  5108.00  5108.00    8239   
2024-05-03 07:00:00-04:00  5108.00  5112.50  5105.25  5107.50   11919   
2024-05-03 08:00:00-04:00  5107.25  5157.00  5106.25  5148.00  122150   
2024-05-03 09:00:00-04:00  5148.00  5161.50  5137.25  5157.25  255102   
2024-05-03 10:00:00-04:00  5157.75  5166.75  5128.00  5132.75  407534   
2024-05-03 11:00:00-04:00  5133.00  5